In [ ]:
##Import Dependencies

In [1]:
import pandas as pd
import pyodbc
import pgeocode
import numpy as np

In [45]:
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
##Connect to the database

In [2]:
username = 'vigrose'
password = 'slytherin10946'

In [3]:
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [ ]:
##Read in all latest insurance addresses

In [4]:
sql_query = \
    """ 
    SELECT DISTINCT
    A.PARTY_ID, 
    K.KEY_VAL AS ME, 
    A.POST_CD_ID,
    A.FROM_DT,
    C.PURPOSE_TYPE_DESC AS ADDR_TYPE, 
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP
    FROM 
    AMAEDW.PARTY_KEY K, AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.POST_CD P, AMAEDW.STATE S 
    WHERE  
    A.THRU_DT IS NULL
    AND
    K.PARTY_ID = A.PARTY_ID 
    AND 
    K.KEY_TYPE_ID = 18 
    AND 
    K.ACTIVE_IND = 'Y'
    AND
    C.PURPOSE_TYPE_ID IN (1)
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    ORDER BY A.FROM_DT DESC;
    """

In [5]:
insurance = pd.read_sql(con=AMAEDW, sql=sql_query)
insurance = insurance.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(insurance.shape)
insurance.head()

(1599453, 11)


,PARTY_ID,ME,POST_CD_ID,FROM_DT,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP
0,2450352,05107840674,15160468,2020-03-18 00:09:46,Billing,5 Medical Park Dr Ste 203,,,Benton,AR,72015
1,2425956,04501990596,15176853,2020-03-18 00:09:46,Billing,4700 Waters Ave,Memorial Health,,Savannah,GA,31404
2,1885558,01222990536,15759865,2020-03-18 00:09:46,Billing,1208 Ella St,,,Anderson,SC,29621
3,1877201,01201000854,19331882,2020-03-18 00:09:46,Billing,2436 Allegheny Ln,,,Watkinsville,GA,30677
4,2050792,02101821113,22092866,2020-03-18 00:09:46,Billing,1600 W 40th Ave,,,Pine Bluff,AR,71603


In [14]:
dhc = pd.read_csv('DHC_March_Delta.csv')

In [35]:
nums =[]
for num in list(dhc['ME_feb']):
    num = num.replace('.0','')
    if len(num)==10:
        num = '0'+num
    elif len(num)==9:
        num = '00'+num
    elif len(num)==8:
        num = '000'+num
    nums.append(num)

In [37]:
dhc['ME'] = nums

In [31]:
dhc_2 = dhc[dhc['ME_feb']!='None']

In [23]:
insurance['ME']

0           05107840674
1           04501990596
2           01222990536
3           01201000854
4           02101821113
               ...     
14216157    01606689957
14227024    01606769926
14237322    02401980221
14307953    00512010695
14308220    01001001422
Name: ME, Length: 1599453, dtype: object

In [40]:
merged = pd.merge(dhc, insurance, on='ME')

In [43]:
merged[['Address_feb','Address_mar','ADDR_1','ADDR_2']]

,Address_feb,Address_mar,ADDR_1,ADDR_2
0,121 Medical Center Dr,Medical Office Building,850 Baxter Blvd,
1,1101 Veterans Dr,830 S Limestone,900 S. Limestone Street,University Of Kentucky
2,531 W College St,c/o PAMC Surgery Department,1725 Loma Rd,
3,8080 Parkway Dr,8080 Parkway Dr,15434 Sky High Rd,
4,13652 Cantara St,13652 Cantara St,13652 Cantara St,
...,...,...,...,...
7943,1200 Pleasant St,1200 Pleasant St,1200 Pleasant St,
7944,2351 Erwin Road,2351 Erwin Rd,15902 Wyandotte St,None
7945,1116 Lucerne Ter,1116 Lucerne Ter,1116 Lucerne Ter,
7946,1400 N. I-35,1400 N Interstate 35,1400 N Interstate 35,"Ut At Austin Dell Em Residency Program, Suite ..."


In [100]:
count = 0
match = []
for row in DHC_MERGE.itertuples():
    if fuzz.ratio(row.Address_mar,row.ADDR_1)>70:
        count += 1
        match.append(True)
    else:
        match.append(False)

In [101]:
DHC_MERGE['Match']=match

In [103]:
DHC_MERGE

,NPI,ME_feb,Additional Office Locations_feb,Phone_Number_feb,Zip Code_feb,State_feb,City_feb,Address_feb,First Name_feb,Physician Name_feb,...,POST_CD_ID_POLO,ADDR_1_POLO,ADDR_2_POLO,ADDR_3_POLO,CITY_POLO,STATE_CD_POLO,ZIP_POLO,FROM_DT_POLO,POST_KEY,Match
0,1538110820,18743010015.0,"Little Rock, AR - Springdale, AR",501.364.1202,72202,AR,Rock,1 Children's Way Little,Felipe,Felipe Medeiros,...,19877248.0,9415 Campus Point Dr,,,La Jolla,CA,92093,2011-05-10 21:29:01.338102,12125265.0,False
1,1548449663,87501000467.0,"Weatherford, TX - Burleson, TX - Fort Worth, TX",469.698.1000,75032,TX,Rockwall,3150 Horizon Rd,Fysal,Fysal Albaalbaki,...,26223335.0,508 S Adams St,Ste 102,,Fort Worth,TX,76104,2016-11-03 21:34:58.038529,20412429.0,False
2,1467637025,24352950056.0,None,914.493.8793,10595,NY,Valhalla,100 Woods Rd,Minghao,Minghao Zhong,...,21793871.0,100 Woods Rd,,,Valhalla,NY,10595,2014-02-27 21:28:10.862490,15576720.0,False
3,1477852713,3545110945.0,None,253.382.8450,98405,WA,Tacoma,1708 Yakima Ave Ste 112,Michael,Michael Lin,...,10682187.0,428 E 72nd St,Ofc 500,,New York,NY,10021,2019-12-22 21:07:40.217374,8078813.0,True
4,1497075626,4802100707.0,"Fort Worth, TX - Cleburne, TX - Stephenville, ...",None,75028,TX,Flower Mound,4400 Long Prairie Rd,Justin,Justin Tran,...,25943158.0,5323 Harry Hines Blvd,Radiology Dept,Univ of Texas Southwestern Med Ctr,Dallas,TX,75390,2015-08-08 17:59:26.815387,20150019.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1164840682,2101141805.0,"Kingston, PA",570.288.3601,18704,PA,Kingston,423 3rd Ave Ste B,Daniel,Daniel Rittenberg,...,25942858.0,HSC T-9 Rm 040,Urology Pgm,SUNY Stony Brook Univ,Stony Brook,NY,11794,2015-08-13 21:51:24.481991,20150154.0,True
1096,1205185345,70425090460.0,None,214.590.8058,75235,TX,Dallas,5201 Harry Hines Blvd,Afaq,Afaq Motiwala,...,20847739.0,3401 N Broad St,Cardiology Ste 945PP,Temple Univ Hosp,Philadelphia,PA,19140,2018-09-21 21:08:51.171414,14290576.0,False
1097,1760869374,1803150601.0,None,515.241.5926,50309,IA,Des Moines,1200 Pleasant St,Sarah,Sarah Francis,...,22156932.0,1400 S Dobson Rd,,,Mesa,AZ,85202,2018-06-22 20:49:51.410292,15437563.0,True
1098,1801271457,515140221.0,None,919.684.8434,27710,NC,Durham,2351 Erwin Road,Asghar,Asghar Haider,...,26708865.0,2020 Gravier St,Eye Ctr Ste B,LSU Health Sciences Ctr New Orleans,New Orleans,LA,70112,2016-08-04 21:30:39.920226,20750201.0,False


In [104]:
DHC_MERGE.columns

Index(['NPI', 'ME_feb', 'Additional Office Locations_feb', 'Phone_Number_feb',
       'Zip Code_feb', 'State_feb', 'City_feb', 'Address_feb',
       'First Name_feb', 'Physician Name_feb', 'Middle Name_feb',
       'Primary Specialty_feb', 'Primary Hospital Affiliation_feb',
       'Last Name_feb', 'Address1_feb', 'ME_mar',
       'Additional Office Locations_mar', 'Phone_Number_mar', 'Zip Code_mar',
       'State_mar', 'City_mar', 'Address_mar', 'First Name_mar',
       'Physician Name_mar', 'Middle Name_mar', 'Primary Specialty_mar',
       'Primary Hospital Affiliation_mar', 'Last Name_mar', 'Address1_mar',
       'New', 'Phone_Update', 'Address_Update', 'ME', 'PARTY_ID', 'POST_CD_ID',
       'FROM_DT', 'ADDR_TYPE', 'ADDR_1', 'ADDR_2', 'ADDR_3', 'CITY',
       'STATE_CD', 'ZIP', 'POST_CD_ID_PPMA', 'ADDR_1_PPMA', 'ADDR_2_PPMA',
       'ADDR_3_PPMA', 'CITY_PPMA', 'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA',
       'POST_CD_ID_POLO', 'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO',
       

In [105]:
DHC_MERGE[DHC_MERGE.Match == True][['Address_feb','Address_mar','Phone_Number_mar']]

,Address_feb,Address_mar,Phone_Number_mar
3,1708 Yakima Ave Ste 112,3100 Ne 28th St,541.994.8114
7,10180 Se Sunnyside Rd,10180 Se Sunnyside Rd,360.256.0556
12,2091 Nostrand Ave,900 W Clairemont Ave,715.717.4121
17,21601 76th Ave W,3322 Us Highway 22,908.378.7227
20,701 Enfield Street,701 Enfield St,860.741.6058
...,...,...,...
1091,515 Delaware Street Se,515 Delaware St Se,612.626.3111
1093,20 York St # Cb-2041,20 York St,203.688.4748
1094,6633 Telephone Rd,1250 16th St,310.319.4698
1095,423 3rd Ave Ste B,423 3rd Ave,570.288.3601


In [51]:
ranked = pd.read_csv('Polo_Addr_Rank_Scored_PPD_DPC_Pop.csv')

In [6]:
insurance_2 = insurance.drop_duplicates(subset = 'PARTY_ID', keep='first')

In [ ]:
#Read all latest PPMA & POLO addresses

In [7]:
sql_query_PPMA = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PP'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
        '''

In [8]:
PPMA = pd.read_sql(con=AMAEDW, sql=sql_query_PPMA)
print(PPMA.shape)
PPMA.head()

(1602813, 9)


,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT
0,11547914,30710349,330 N Wabash Ave,Fl 40,,Chicago,IL,60611,2020-03-17 21:08:32.696113
1,15016645,30710394,5535 Hartsdale Dr,,,Jackson,MS,39211,2020-03-17 21:08:32.691398
2,6162817,30710336,142 Main St,Apt 2B,,Mineola,NY,11501,2020-03-17 21:08:32.689417
3,6166527,30710335,121 S Lake Ave,Apt 3,,Albany,NY,12208,2020-03-17 21:08:32.674965
4,13432309,30710334,506 6th St,"East Pavilion, 4th floor",Dept of OBGYN,Brooklyn,NY,11215,2020-03-17 21:08:32.666305


In [9]:
sql_query_POLO = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
    '''

In [10]:
POLO = pd.read_sql(con=AMAEDW, sql=sql_query_POLO)
print(POLO.shape)
POLO.head()

(1163743, 10)


,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT,POST_KEY
0,2149165,30710407,16715 Aurora Ave N,Ste 102,,Shoreline,WA,98133,2020-03-17 21:08:32.411316,24434990
1,1833025,30710354,316 7th Ave N,,,Safety Harbor,FL,34695,2020-03-17 21:08:29.499439,24434888
2,2965319,30710388,98 Cope Creek Rd,,,Sylva,NC,28779,2020-03-17 21:08:29.396482,24434399
3,2973188,28904613,596 Jericho Tpke,,,Syosset,NY,11791,2020-03-17 21:08:29.190080,22478034
4,1836757,21895156,6181 N Federal Hwy,,,Ft Lauderdale,FL,33308,2020-03-17 21:08:29.093276,15175531


In [53]:
POLO = POLO.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(POLO.shape)
PPMA= PPMA.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(PPMA.shape)

(1163740, 10)
(1602779, 9)


In [55]:
##Read latest PPD
ppd = pd.read_csv('ppd_data_20200314.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
dhc = pd.read_csv('DHC_Data.csv')

In [39]:
dhc = dhc.fillna('None')

In [74]:
nums =[]
for num in MES:
    if num!= 'None':
        num = str(int(num))
    if len(num)==10:
        num = '0'+num
    elif len(num)==9:
        num = '00'+num
    elif len(num)==8:
        num = '000'+num
    nums.append(num)

In [45]:
dhc.ME=nums

In [47]:
pd.merge(insurance_2, dhc, on='ME').to_csv('dhc_insurance.csv', index=False)

In [ ]:
##Merge PPMA, POLO, PPD, and Insurance by ME and PARTY_ID

In [69]:
MES = list(ppd['ME'].astype(str))

In [78]:
len(ppd)

1262355

In [79]:
insurance_2 = insurance[insurance.ME.isin(nums)]

In [80]:
ppma_polo = pd.merge(PPMA, POLO, on = 'PARTY_ID', how= 'outer', suffixes = ('_PPMA', '_POLO'))

In [81]:
ppd_insurance_2 = pd.merge(insurance_2, ppma_polo, on = 'PARTY_ID')

In [19]:
INSURANCE_2 = ppd_insurance_2.drop(columns=['PARTY_ID','ME', 'POST_CD_ID'])

In [20]:
INSURANCE_2.to_csv('insurance_current.csv')

In [9]:
INSURANCE.to_csv('insurance_historical.gz', compression='gzip')

In [ ]:
PPD.shape

In [ ]:
ppd_insurance.shape

In [ ]:
insurance_polo = pd.merge(ppd_insurance, POLO, on = 'PARTY_ID', suffixes = ('_insurance', '_POLO'))

In [ ]:
insurance_ppma = pd.merge(ppd_insurance, PPMA, on = 'PARTY_ID', suffixes = ('_insurance', '_PPMA'))

In [ ]:
insurance_ppma_polo = pd.merge(ppd_insurance, ppma_polo, on = 'PARTY_ID')

In [ ]:
insurance_ppma_polo.shape

In [ ]:
insurance_ppma_polo = insurance_ppma_polo.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(insurance_ppma_polo.shape)

In [ ]:
##Summarize the address differences and generate a table of mismat

In [ ]:
#Define summarizing function (PPMA)
def sum_it_up_ppma(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT_insurance > row.FROM_DT_PPMA:
            if row.POST_CD_ID_PPMA != row.POST_CD_ID_insurance:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
                new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
                new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
                new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                new_dict['ZIP_insurance'] = row.ZIP_insurance
                new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                new_dict['CITY_insurance'] = row.CITY_insurance
                new_dict['CITY_PPMA'] = row.CITY_PPMA
                new_dict['DATE_insurance'] = row.FROM_DT_insurance
                new_dict['DATE_PPMA'] = row.FROM_DT_PPMA
                new_dict['STATE_insurance'] = row.STATE_CD_insurance
                new_dict['STATE_PPMA'] = row.STATE_CD_PPMA
                dict_list.append(new_dict)

    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs do not match 
    ''')
    return(dict_list)

In [ ]:
#Define summarizing function (POLO)
def sum_it_up_polo(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT_insurance > row.FROM_DT_POLO:
            if row.POST_CD_ID_POLO == row.POST_CD_ID_insurance:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
                new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
                new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
                new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                new_dict['ZIP_insurance'] = row.ZIP_insurance
                new_dict['ZIP_POLO'] = row.ZIP_POLO
                new_dict['CITY_insurance'] = row.CITY_insurance
                new_dict['CITY_POLO'] = row.CITY_POLO
                new_dict['DATE_insurance'] = row.FROM_DT_insurance
                new_dict['DATE_POLO'] = row.FROM_DT_POLO
                new_dict['STATE_insurance'] = row.STATE_CD_insurance
                new_dict['STATE_POLO'] = row.STATE_CD_POLO
                dict_list.append(new_dict)

    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs match 
    ''')
    return(dict_list)

In [ ]:
#Define summarizing function (ALL)
def sum_it_up(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT > row.FROM_DT_PPMA and row.FROM_DT > row.FROM_DT_POLO:
            if row.POST_CD_ID_POLO != row.POST_CD_ID and row.POST_CD_ID != row.POST_CD_ID_PPMA:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1
                new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                new_dict['ADDR_2_insurance'] = row.ADDR_2
                new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                new_dict['ADDR_3_insurance'] = row.ADDR_3
                new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                new_dict['ZIP_insurance'] = row.ZIP
                new_dict['ZIP_POLO'] = row.ZIP_POLO
                new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                new_dict['CITY_insurance'] = row.CITY
                new_dict['CITY_POLO'] = row.CITY_POLO
                new_dict['CITY_PPMA'] = row.CITY_PPMA
                new_dict['DATE_insurance'] = row.FROM_DT
                new_dict['DATE_POLO'] = row.FROM_DT_POLO
                new_dict['DATE_PPMA'] = row.FROM_DT_PPMA
                new_dict['STATE_insurance'] = row.STATE_CD
                new_dict['STATE_POLO'] = row.STATE_CD_POLO
                new_dict['STATE_PPMA'] = row.STATE_CD_PPMA
                dict_list.append(new_dict)

                if row.ADDR_1 != row.ADDR_1_PPMA and row.ADDR_1 != row.ADDR_1_POLO:
                    add_count +=1
                if row.CITY != row.CITY_PPMA and row.CITY != row.CITY_POLO:
                    city_count +=1
                if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                    zip_count +=1
                if row.STATE_CD != row.STATE_CD_PPMA and row.STATE_CD != row.STATE_CD_POLO:
                    state_count +=1
    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs do not match 
    {add_count/cd_count*100}% ({add_count}) addresses do not match
    {zip_count/cd_count * 100}% ({zip_count})zipcodes do not match
    {state_count/cd_count * 100}% ({state_count})states do not match
    {city_count/cd_count * 100}% ({city_count})cities do not match
    ''')
    return(dict_list)

In [ ]:
insurance_mismatch = sum_it_up(insurance_ppma_polo)
insurance_mismatch_df = pd.DataFrame(insurance_mismatch)

In [ ]:
polo_match = sum_it_up_polo(insurance_polo)
polo_match_df = pd.DataFrame(polo_match)

In [ ]:
ppma_mismatch = sum_it_up_ppma(insurance_ppma)
ppma_mismatch_df = pd.DataFrame(ppma_mismatch)

In [ ]:
pd.merge(ppma_match_df, polo_match_df, on = ['ME','ADDR_1_insurance','ADDR_2_insurance','ADDR_3_insurance', 'CITY_insurance', 'DATE_insurance', 'STATE_insurance', 'ZIP_insurance'])

In [ ]:
#Define summarizing function
def narrow_it_down(df, address_type):
    dict_list = []
    for row in df.itertuples():
        if address_type == 'PPMA'
            if row.STATE_insurance != row.STATE_PPMA:
                if row.CITY_insurance != row.CITY_PPMA:
                    if row.ADDR_1_insurance != row.ADDR_1_PPMA:
                        if row.ZIP_insurance != row.ZIP_PPMA:
                            new_dict = {}
                            new_dict['ME'] = row.ME
                            new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance

                            new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                            new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance

                            new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                            new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance

                            new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                            new_dict['ZIP_insurance'] = row.ZIP_insurance

                            new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                            new_dict['CITY_insurance'] = row.CITY_insurance

                            new_dict['CITY_PPMA'] = row.CITY_PPMA
                            new_dict['DATE_insurance'] = row.DATE_insurance

                            new_dict['DATE_PPMA'] = row.DATE_PPMA
                            new_dict['STATE_insurance'] = row.STATE_insurance

                            new_dict['STATE_PPMA'] = row.STATE_PPMA
                            dict_list.append(new_dict)
                            
        if address_type == 'POLO'
            if row.STATE_insurance != row.STATE_POLO:
                if row.CITY_insurance != row.CITY_POLO:
                    if row.ADDR_1_insurance != row.ADDR_1_POLO:
                        if row.ZIP_insurance != row.ZIP_POLO:
                            new_dict = {}
                            new_dict['ME'] = row.ME
                            new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance

                            new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                            new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance

                            new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                            new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance

                            new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                            new_dict['ZIP_insurance'] = row.ZIP_insurance

                            new_dict['ZIP_POLO'] = row.ZIP_POLO
                            new_dict['CITY_insurance'] = row.CITY_insurance

                            new_dict['CITY_POLO'] = row.CITY_POLO
                            new_dict['DATE_insurance'] = row.DATE_insurance

                            new_dict['DATE_POLO'] = row.DATE_POLO
                            new_dict['STATE_insurance'] = row.STATE_insurance

                            new_dict['STATE_POLO'] = row.STATE_POLO
                            dict_list.append(new_dict)          
    NEW_DF=pd.DataFrame(dict_list)
    return(NEW_DF)

In [94]:
#Define summarizing function
def narrow_it_down(df):
    match_list = []
    for row in df.itertuples():
        match = True
        if row.FROM_DT > row.FROM_DT_POLO and row.FROM_DT > row.FROM_DT_PPMA:
            if row.STATE_CD != row.STATE_CD_PPMA and row.STATE_CD != row.STATE_CD_POLO:
                if row.CITY != row.CITY_PPMA and row.CITY != row.CITY_POLO:
                    if row.ADDR_1 != row.ADDR_1_PPMA and row.ADDR_1!= row.ADDR_1_POLO:
                        if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                            match = False
        match_list.append(match)
    df['Match'] = match_list
    return(df)

In [140]:
def narrow_it_down_2(df):
    match_list = []
    for row in df.itertuples():
        match = True
        if row.FROM_DT > row.FROM_DT_POLO and row.FROM_DT > row.FROM_DT_PPMA:
            if row.ADDR_1 != row.ADDR_1_PPMA and row.ADDR_1!= row.ADDR_1_POLO:
                if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                    match = False
        match_list.append(match)
    df['Match_2'] = match_list
    return(df)

In [93]:
ppd_insurance_2.dtypes

PARTY_ID                    int64
ME                         object
POST_CD_ID                  int64
FROM_DT            datetime64[ns]
ADDR_TYPE                  object
ADDR_1                     object
ADDR_2                     object
ADDR_3                     object
CITY                       object
STATE_CD                   object
ZIP                        object
POST_CD_ID_PPMA           float64
ADDR_1_PPMA                object
ADDR_2_PPMA                object
ADDR_3_PPMA                object
CITY_PPMA                  object
STATE_CD_PPMA              object
ZIP_PPMA                   object
FROM_DT_PPMA       datetime64[ns]
POST_CD_ID_POLO           float64
ADDR_1_POLO                object
ADDR_2_POLO                object
ADDR_3_POLO                object
CITY_POLO                  object
STATE_CD_POLO              object
ZIP_POLO                   object
FROM_DT_POLO       datetime64[ns]
POST_KEY                  float64
Match                        bool
dtype: object

In [152]:
narrowed_2 = narrow_it_down_2(ppd_insurance_2)

In [107]:
narrowed.to_csv('Insurance_Select.csv', index=False)

In [109]:
usage = pd.read_csv('Add_Usage.csv')

In [158]:
DIFF_2 = narrowed_2[narrowed_2['Match_2']==False]

In [127]:
len

,PARTY_ID,ME,POST_CD_ID,FROM_DT,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,...,POST_CD_ID_POLO,ADDR_1_POLO,ADDR_2_POLO,ADDR_3_POLO,CITY_POLO,STATE_CD_POLO,ZIP_POLO,FROM_DT_POLO,POST_KEY,Match
1,2425956,04501990596,15176853,2020-03-18 00:09:46.000000,Billing,4700 Waters Ave,Memorial Health,,Savannah,GA,...,16013479.0,171 Ashley Ave,,,Charleston,SC,29425,2006-10-13 00:01:12.825717,8337958.0,False
5,2256798,02834911241,22181245,2020-03-18 00:09:46.000000,Billing,6150 W Layton Ave,,,Greenfield,WI,...,20722272.0,47111 Monroe St,John F Kennedy Memorial Hospital,,Indio,CA,92201,2018-06-08 20:57:23.541024,14127502.0,False
11,3519157,03843071049,26255432,2020-03-18 00:09:46.000000,Billing,8901 Carti Way,,,Little Rock,AR,...,26578458.0,7400 Fannin St,Ste 1295,,Houston,TX,77054,2016-06-08 23:11:57.642529,20649454.0,False
16,2856691,04707020640,30711330,2020-03-18 00:09:46.000000,Billing,2100 Madison Ave,Gateway Medical Center,None,Granite City,IL,...,28415824.0,300 Carson St,Ste A,,Jonesboro,AR,72401,2018-01-19 20:50:44.089924,22139940.0,False
21,2033304,01720800906,30711353,2020-03-18 00:09:46.000000,Billing,1734 Cedar Creek Rd,Capital City Ortho/Sports Med,None,Glenville,NC,...,14908126.0,5555 Peachtree Dunwoody Rd,Ste 101 Medical Qtrs,,Atlanta,GA,30342,2005-12-16 20:36:47.573450,1157154.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246098,2729969,64901570653,12690692,2016-10-17 15:35:58.050344,Billing,13535 Detroit Ave,,,Cleveland,OH,...,9348571.0,3734 Whittier Blvd,,,Los Angeles,CA,90023,2005-12-16 22:07:02.880271,3157198.0,False
1250772,2460209,05606771381,13662508,2016-10-17 15:14:44.452244,Billing,72 Cambridge Rd,,,Montclair,NJ,...,10499403.0,153 W 11th St,,,New York,NY,10011,2005-12-16 21:50:24.166674,1040539.0,False
1251027,2447898,05104490273,11445686,2016-10-17 15:10:37.355970,Billing,1594 Pinetree Dr,,,Pittsburgh,PA,...,14318759.0,6606 W Broad St,4th Fl,,Richmond,VA,23230,2015-10-01 22:32:13.640477,2436195.0,False
1251097,2615219,05101540385,14318759,2016-10-17 15:09:28.458873,Billing,6606 W Broad St,4th Fl,,Richmond,VA,...,9172720.0,1540 5th Ave,,,San Rafael,CA,94901,2005-12-16 21:41:41.102973,2435669.0,False


In [116]:
nums =[]
for num in list(usage['ME'].astype(str)):
    num = num.replace('.0','')
    if len(num)==10:
        num = '0'+num
    elif len(num)==9:
        num = '00'+num
    elif len(num)==8:
        num = '000'+num
    nums.append(num)
usage['ME']=nums

In [117]:
DHC_MERGE = pd.merge(dhc, usage, on='ME')

In [159]:
dhc_merge_3 = pd.merge(DIFF_2, dhc, on='ME')

In [160]:
dhc_merge_3 = dhc_merge_3.fillna('None')

In [134]:
dhc_merge_2.columns

Index(['PARTY_ID', 'ME', 'POST_CD_ID', 'FROM_DT', 'ADDR_TYPE', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE_CD', 'ZIP', 'POST_CD_ID_PPMA',
       'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA', 'CITY_PPMA',
       'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POST_CD_ID_POLO',
       'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO', 'CITY_POLO',
       'STATE_CD_POLO', 'ZIP_POLO', 'FROM_DT_POLO', 'POST_KEY', 'Match', 'NPI',
       'ME_feb', 'Additional Office Locations_feb', 'Phone_Number_feb',
       'Zip Code_feb', 'State_feb', 'City_feb', 'Address_feb',
       'First Name_feb', 'Physician Name_feb', 'Middle Name_feb',
       'Primary Specialty_feb', 'Primary Hospital Affiliation_feb',
       'Last Name_feb', 'Address1_feb', 'ME_mar',
       'Additional Office Locations_mar', 'Phone_Number_mar', 'Zip Code_mar',
       'State_mar', 'City_mar', 'Address_mar', 'First Name_mar',
       'Physician Name_mar', 'Middle Name_mar', 'Primary Specialty_mar',
       'Primary Hospital Affiliation_m

In [161]:
COUNT = 0
dhc_match = []
for row in dhc_merge_3.itertuples():
    if fuzz.ratio(row.ADDR_1, row.Address_mar)>70:
        COUNT += 1 
        dhc_match.append(True)
    else:
        dhc_match.append(False)

In [163]:
dhc_merge_3['DHC_MATCH']=dhc_match

In [139]:
dhc_merge_2.to_csv('SAMPLE_WORK.csv',index=False)

In [ ]:
insurance_no_matches  = narrow_it_down(insurance_mismatch_df)

In [ ]:
no_matches = narrow_it_down(mismatch_df)

In [ ]:
def get_zip_distance(df, DIST=pgeocode.GeoDistance('US')):
    distance = DIST.query_postal_code(df[],df[])*0.621371
    return(distance)

In [ ]:
#Define function to get distances between zipcodes in dataframe
def get_zip_distances(df):    
    distances = []
    TOTAL = df.shape[0]
    print(TOTAL)
    count =0
    DIST=pgeocode.GeoDistance('US')
    dict_list = []
    num_list =[]
    for row in df.itertuples():
        try:
            new_dict={}
            new_dict["ME"]=row.ME
            count += 1
            if int(count/TOTAL*100) in list(range(1,100)) and int(count/TOTAL*100) not in num_list:
                num_list.append(int(count/TOTAL*100))
                print (f'{int(count/TOTAL*100)}% done!')
            try:
                mf = int(row.ZIP_POLO)
    #             print(mf)
                ins = int(row.ZIP_insurance)
    #             print(ins)
            except:
                mf='nan'
                ins='nan'
        #     print(f'{mf} {ins}')
            try:
                ZIP_distance = get_zip_distance(mf,ins)
    #             print(ZIP_distance)
                if np.isnan(ZIP_distance)==False:
                    distances.append(ZIP_distance)     
                else:
                    print(f'{count}: {row.ME} \nMasterfile:{mf} \nInsurance:{ins}')
            except:
                ZIP_distance = 'nan'
                print(f'{count}: {row.ME} \nMasterfile:{mf} \nInsurance:{ins}')
            new_dict['ZIP_distance']=ZIP_distance
            dict_list.append(new_dict)
        # print(distances2)
        except KeyboardInterrupt:
            print("Interrupting...")
            return(dict_list, distances)
        try:
            avg = sum(distances)/len(distances)
        except:
            avg=0
    print(f'Average distance between insurance and masterfile addressses is {avg} miles.')
    return(dict_list, distances)

In [ ]:
zip_dicts_polo, zip_dists_polo = get_zip_distances(no_matches)

In [ ]:
def get_zip_distance(zip1,zip2):
    zip1 = clean_zipcode(zip1)
    zip2 = clean_zipcode(zip2)
    Distance = DIST.query_postal_code(zip1, zip2)*0.621371
    return(Distance)

In [ ]:
zippos = zip_dicts+zip_dicts2 + zip_dicts3
pd.DataFrame(zippos)

In [ ]:
no_matches.shape[0]

In [ ]:
def clean_zipcode(zipcode):
    zipcode=str(zipcode)
    zipcode = zipcode.replace(" ","")
    zipcode = zipcode.replace(".0","")
    if len(zipcode)==3:
        zipcode = "00"+zipcode
    if len(zipcode)==4: 
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return(zipcode)

In [ ]:
no_matches.to_csv('no_matches.csv')

In [ ]:
polo_dist = pd.DataFrame(zip_dicts_polo)

In [ ]:
ppma_dist=pd.DataFrame(zippos)

In [ ]:
no_matches = no_matches.drop(columns=['POLO_distance'])
with_zips = pd.merge(no_matches, polo_dist, on='ME', how='outer')

In [ ]:
with_zips = pd.merge(with_zips, ppma_dist, on='ME', how='outer', suffixes = ('_polo', '_ppma'))

In [ ]:
with_zips['ZIP_distance_polo'] = with_zips['ZIP_distance_polo'].astype(float)
with_zips['ZIP_distance_ppma'] = with_zips['ZIP_distance_ppma'].astype(float)

In [ ]:
far_places = with_zips[(with_zips['ZIP_distance_polo']>300) & (with_zips['ZIP_distance_ppma']>300)]

In [ ]:
far_places[['DATE_POLO', 'DATE_PPMA', 'DATE_insurance']]

In [ ]:
all_addresses[['PARTY_ID','ME']]

In [ ]:
small_table = all_addresses[['PARTY_ID','POST_CD_ID', 'ME']]

In [ ]:
with_ids = pd.merge(small_table, far_places, on='ME')

In [ ]:
pd.merge(with_ids, long_time, on=['PARTY_ID', 'POST_CD_ID'])

In [ ]:
with_ids.to_csv('Insurance_Updates.csv', index=False)

In [ ]:
far_places

In [ ]:
ppma_match_df.to_csv('PPMA_Insurance_Matches.csv',index=False)

In [ ]:
ppma_mismatch_df.to_csv('PPMA_Insurance_POST_CD_ID_Mismatches.csv',index=False)

In [ ]:
ppma_extreme_mismatches = narrow_it_down_ppma(ppma_mismatch_df)


In [ ]:
print(f'{ppma_extreme_mismatches.shape} < {ppma_mismatch_df.shape}')

In [ ]:
ppma_extreme_mismatches.to_csv('PPMA_Insurance_Complete_Address_Mismatches.csv',index=False)

In [ ]:
insurance_no_matches.to_csv('PPMA_POLO_Insurance_Complete_Mismatches.csv',index=False)

In [ ]:
insurance_no_matches.dtypes

In [ ]:
ppma_extreme_mismatches['STATUS']='BAD'
ppma_match_df['STATUS']='GOOD'
all_ppma_insurance = pd.concat([ppma_extreme_mismatches, ppma_match_df])

In [ ]:
all_ppma_insurance.groupby('STATUS').count()

In [ ]:
all_ppma_insurance.to_csv('Insurance_PPMA_2019_10_16.csv')

In [ ]:
[
        'ME',
        'RECORD_ID',
        'UPDATE_TYPE',
        'ADDRESS_TYPE',
        'MAILING_NAME',
        'LAST_NAME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'SUFFIX',
        'MAILING_LINE_1',
        'MAILING_LINE_2',
        'CITY',
        'STATE',
        'ZIP',
        'SECTOR',
        'CARRIER_ROUTE',
        'ADDRESS_UNDELIVERABLE_FLAG',
        'FIPS_COUNTY',
        'FIPS_STATE',
        'PRINTER_CONTROL_CODE',
        'PC_ZIP',
        'PC_SECTOR',
        'DELIVERY_POINT_CODE',
        'CHECK_DIGIT',
        'PRINTER_CONTROL_CODE_2',
        'REGION',
        'DIVISION',
        'GROUP',
        'TRACT',
        'SUFFIX_CENSUS',
        'BLOCK_GROUP',
        'MSA_POPULATION_SIZE',
        'MICRO_METRO_IND',
        'CBSA',
        'CBSA_DIV_IND',
        'MD_DO_CODE',
        'BIRTH_YEAR',
        'BIRTH_CITY',
        'BIRTH_STATE',
        'BIRTH_COUNTRY',
        'GENDER',
        'TELEPHONE_NUMBER',
        'PRESUMED_DEAD_FLAG',
        'FAX_NUMBER',
        'TOP_CD',
        'PE_CD',
        'PRIM_SPEC_CD',
        'SEC_SPEC_CD',
        'MPA_CD',
        'PRA_RECIPIENT',
        'PRA_EXP_DT',
        'GME_CONF_FLG',
        'FROM_DT',
        'TO_DT',
        'YEAR_IN_PROGRAM',
        'POST_GRADUATE_YEAR',
        'GME_SPEC_1',
        'GME_SPEC_2',
        'TRAINING_TYPE',
        'GME_INST_STATE',
        'GME_INST_ID',
        'MEDSCHOOL_STATE',
        'MEDSCHOOL_ID',
        'MEDSCHOOL_GRAD_YEAR',
        'NO_CONTACT_IND',
        'NO_WEB_FLAG',
        'PDRP_FLAG',
        'PDRP_START_DT',
        'POLO_MAILING_LINE_1',
        'POLO_MAILING_LINE_2',
        'POLO_CITY',
        'POLO_STATE',
        'POLO_ZIP',
        'POLO_SECTOR',
        'POLO_CARRIER_ROUTE',
        'MOST_RECENT_FORMER_LAST_NAME',
        'MOST_RECENT_FORMER_MIDDLE_NAME',
        'MOST_RECENT_FORMER_FIRST_NAME',
        'NEXT_MOST_RECENT_FORMER_LAST',
        'NEXT_MOST_RECENT_FORMER_MIDDLE',
        'NEXT_MOST_RECENT_FORMER_FIRST'
    ]
PPD_countries = ppd[['ME','MAILING_NAME','BIRTH_YEAR','BIRTH_CITY','BIRTH_STATE','BIRTH_COUNTRY']]

In [ ]:
PPD_countries = PPD_countries.dropna(subset=['BIRTH_CITY'])

In [ ]:
PPD_counts = PPD_countries.groupby('BIRTH_COUNTRY').nunique()

In [ ]:
ppd_no_us = PPD_countries[PPD_countries['BIRTH_COUNTRY']!='US1']

In [ ]:
ppd_no_us

In [ ]:
import geonamescache as geo

In [ ]:
city_list = geo.GeonamesCache().get_cities()

In [ ]:
city_df = pd.DataFrame(city_list).transpose()

In [ ]:
city_df['name'] = city_df['name'].str.upper()

In [ ]:
city_trunc = city_df[['countrycode', 'name']]

In [ ]:
city_info = pd.merge(city_trunc, ppd_no_us, left_on = 'name', right_on = 'BIRTH_CITY', how = 'right')

In [ ]:
city_info = city_info.fillna('None')
nones = city_info[city_info['BIRTH_COUNTRY']=='None']

In [ ]:
nones.groupby('countrycode').count()

In [ ]:
# nones = nones.fillna('None')
potential = nones[(nones['countrycode']=='None') & (nones['BIRTH_CITY']!='UNKNOWN') & (nones['BIRTH_COUNTRY']=='None')]
potential.groupby('BIRTH_CITY').count().sort_values(by=['countrycode'], ascending=False)

In [ ]:
ppd_no_us = ppd_no_us.replace('PUSAN', 'BUSAN')

In [ ]:
ppd_no_us[ppd_no_us['BIRTH_CITY']=='BUSAN']

In [ ]:
city_df.sort_values(by=['name'])

In [ ]:
city_trunc.to_csv('Cities.csv')

In [ ]:
city_info.to_csv('Country_Work.csv')

In [ ]:
full = pd.read_csv('Insurance_PPMA_2019_10_16.csv')

In [ ]:
full['ME'] = full['ME'].astype('str')

In [ ]:
full.dtypes

In [ ]:
PPMA.dtypes

In [ ]:
new_PPMA_2 = pd.merge(full, PPMA, left_on=('ME'), right_on=('ME'))

In [ ]:
PPMA= PPMA.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(PPMA.shape)

In [ ]:
PPMA.dtypes

In [ ]:
full.shape

In [ ]:
PPMA.shape

In [ ]:
new_PPMA.drop_duplicates('ME')

In [ ]:
new_PPMA.to_csv('Insurance_PPMA_2019_10_21.csv')

In [ ]:
new_PPMA.groupby('STATUS').count()

In [ ]:
new_PPMA_2.drop_duplicates(subset='ME')

In [ ]:
PPMA['ME'] = PPMA['ME'].apply(lambda x: ('0000' + str(x))[-11:]) 

In [ ]:
full['ME'] = full['ME'].apply(lambda x: ('0000' + str(x))[-11:]) 

In [ ]:
new_PPMA_2.to_csv('Insurance_PPMA_2019_10_21.csv')

In [15]:
insurance_2.shape

(1596297, 11)

In [17]:
INSURANCE_2.shape

(12811241, 12)

In [21]:
INSURANCE_2.shape

(1257687, 12)

In [165]:
dhc_merge_3[dhc_merge_3.DHC_MATCH==True].to_csv('Insurance_Sample_2.csv',index=False)

In [167]:
test = dhc_merge_3[dhc_merge_3.DHC_MATCH==True]

In [168]:
test.columns

Index(['PARTY_ID', 'ME', 'POST_CD_ID', 'FROM_DT', 'ADDR_TYPE', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE_CD', 'ZIP', 'POST_CD_ID_PPMA',
       'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA', 'CITY_PPMA',
       'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POST_CD_ID_POLO',
       'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO', 'CITY_POLO',
       'STATE_CD_POLO', 'ZIP_POLO', 'FROM_DT_POLO', 'POST_KEY', 'Match',
       'Match_2', 'NPI', 'ME_feb', 'Additional Office Locations_feb',
       'Phone_Number_feb', 'Zip Code_feb', 'State_feb', 'City_feb',
       'Address_feb', 'First Name_feb', 'Physician Name_feb',
       'Middle Name_feb', 'Primary Specialty_feb',
       'Primary Hospital Affiliation_feb', 'Last Name_feb', 'Address1_feb',
       'ME_mar', 'Additional Office Locations_mar', 'Phone_Number_mar',
       'Zip Code_mar', 'State_mar', 'City_mar', 'Address_mar',
       'First Name_mar', 'Physician Name_mar', 'Middle Name_mar',
       'Primary Specialty_mar', 'Primary Hospital Af

In [170]:
count_1 = 0
for row in test.itertuples():
    if row.CITY != row.CITY_POLO and row.CITY != row.CITY_PPMA:
        count_1 += 1

In [171]:
count_1

1111

In [172]:
test[test.Phone_Number_mar != 'None']

,PARTY_ID,ME,POST_CD_ID,FROM_DT,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,...,Physician Name_mar,Middle Name_mar,Primary Specialty_mar,Primary Hospital Affiliation_mar,Last Name_mar,Address1_mar,New,Phone_Update,Address_Update,DHC_MATCH
0,3591098,05104090984,14996656,2020-03-18 00:04:36.000000,Billing,4301 W Markham St,Slot 503,,Little Rock,AR,...,Tina Ipe,S,Pathology,Houston Methodist Hospital (FKA the Methodist ...,Ipe,None,True,True,True,True
10,2450580,04901960572,20329300,2020-03-14 04:13:13.000000,Billing,51600 Huntington Rd,La Pine Community Health Center,,La Pine,OR,...,Martin Smart,L,Family Practice,"St Charles Health System - Bend (Bend, OR)",Smart,None,True,True,True,True
13,2590306,04804910625,21506696,2020-03-14 04:13:13.000000,Billing,3803 S Bascom Ave,Ste 204,,Campbell,CA,...,Bradley Hill,B.,Surgery - Vascular Surgery,"Good Samaritan Hospital (San Jose, CA)",Hill,None,False,True,True,True
14,1845538,00102880161,22127673,2020-03-14 04:13:13.000000,Billing,115 E Brooklyn St,P O Box 916,,Linden,TN,...,William Bush,Brent,Internal Medicine,"Lawton Indian Hospital (Lawton, OK)",Bush,None,True,True,True,True
17,4627590,74811050087,25898736,2020-03-14 04:13:13.000000,Billing,900 W Clairemont Ave,Hshs Sacred Heart Hospital,,Eau Claire,WI,...,Katrina Victoria,V,Internal Medicine,"St Barnabas Hospital (Bronx, NY)",Victoria,None,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,2250527,02834990078,21860862,2016-10-19 16:36:59.636308,Billing,301 S 7th Ave,Ste 135,,West Reading,PA,...,Daniel O'shea,J,Radiology - Diagnostic Radiology,"Reading Hospital (West Reading, PA)",O'shea,Suite 135,False,True,False,True
3863,2862034,00518021265,21727893,2016-10-19 16:33:25.242108,Billing,400 Craven Rd,Kaiser San Marcos Medical Office,,San Marcos,CA,...,Herbert Yue,None,Internal Medicine,Palomar Medical Center Downtown Escondido (FKA...,Yue,None,False,True,False,True
3864,1806540,00502981064,21500944,2016-10-19 14:33:42.873766,Billing,732 N Broadway,Pediatrics,,Escondido,CA,...,Monte Klaudt,R,Pediatric Medicine,Kaiser Permanente - Zion Medical Center (FKA S...,Klaudt,None,False,True,False,True
3866,2144908,02105861179,20019128,2016-10-19 09:04:34.739703,Billing,1255 W Arrow Hwy,San Dimas Medical Offices,,San Dimas,CA,...,John Paschal,None,Ophthalmology,Kaiser Permanente - Baldwin Park Medical Cente...,Paschal,None,False,True,False,True


In [173]:
len(test)

1237